#  Setup stuff: Connectivity

In [3]:

from katana import remote
from katana.remote import import_data

my_client = remote.Client()

print(my_client)


VERBOSE: ../../../../external/katana/libgalois/include/katana/AsyncPool.h:152: 18860: async pool tids: 140388424836864, 140388422735616
VERBOSE: ../../../../external/katana/libgalois/include/katana/AsyncPool.h:152: 18860: async pool tids: 140388420634368, 140388416435968
VERBOSE: ../../../../external/katana/libgalois/include/katana/AsyncPool.h:152: 18860: async pool tids: 140388412237568, 140388403844864
VERBOSE: ../../../../external/katana/libgalois/include/katana/AsyncPool.h:152: 18860: async pool tids: 140388395452160, 140388378670848
VERBOSE: ../../../../external/katana/libgalois/include/katana/AsyncPool.h:152: 18860: async pool tids: 140388361889536, 140388359788288


In [4]:
NUM_PARTITIONS = 3
   #
DB_NAME        = "my_db"
GRAPH_NAME     = "my_graph"

print("--")


--


In [5]:
#  CONNECT TO GRAPH

for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      my_graph=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
         #
      break

print(my_graph)


<_Graph my_graph, BngAFupup4DduWF6BxHEqh8xwyN6iymwp7bX4uEfNico, 1>


#  Setup stuff: Debugging

In [ ]:
display(my_graph.num_nodes())
display(my_graph.num_edges())

#  Outputted values,
#
#  24
#  42


In [ ]:

l_result = my_graph.query("""

   MATCH (n) - [ r ] -> (m)
   RETURN n, r, m
   
   """, contextualize=True)

l_result.view()


<div> 
<img src="./01_Images/Result_01.png" alt="Drawing" style="width: 1800px;"/>
</div>


In [ ]:

l_result = my_graph.query("""

   MATCH (n) 
   WITH DISTINCT LABELS(n) AS temp, COUNT(n) AS tempCnt
   UNWIND temp AS label
   RETURN label, SUM(tempCnt) AS cnt
   ORDER BY label
   
   """, contextualize=True)

l_result.view()


# display(print(l_result))
# 
#  Outputted values
#
#     cnt        label
#  0    2  Dist_Center
#  1    4         Item
#  2    8        Order
#  3    4       Person
#  4    2     Reviewer
#  5    4        Store



In [ ]:

l_result = my_graph.query("""

   MATCH (n) - [r] -> (m) 
   WITH DISTINCT TYPE(r) AS temp, COUNT(r) AS tempCnt
   RETURN temp, tempCnt
   ORDER BY temp

   """, contextualize=True)

l_result.view()


# display(print(l_result))
# 
#  Outputted values,
#
#                     temp  tempCnt
#  0          COMPRISED_OF        8
#  1  FOUND_AT_DIST_CENTER        6
#  2        FOUND_AT_STORE       12
#  3                PLACED        8
#  4           REVIEWED_BY        8



#  Analytics: Why we are here

In [ ]:

#  Setting display width 

import pandas as pd
   #
pd.set_option("display.width", 480)

print("--")


In [ ]:

l_result = my_graph.query("""

   MATCH (n: Person) - [r] -> (m) 
   WHERE
      m.city = 'Houston'
   RETURN *

   """, contextualize=True)

l_result.view()


# display(print(l_result))
# 
#  Outputted value,
#
#  Empty DataFrame
#  Columns: []
#  Index: []



In [ ]:

l_result = my_graph.query("""

   MATCH (n: Person) - [r *] -> (m) 
   WHERE
      m.city = 'Houston'
   RETURN n, r, m

   """, contextualize=True)

l_result.view()


# display(print(l_result))
# 
#  Outputted value,
#
#  OperationError: FZSoWndtUBvHNquRkos1tkw5DS9CwvH6NF4X5Ady2WfZ-43CxYsTPwbyPAsoQQ backtrace (QueryClient.cpp:575):
#  opgraph check failed (OpGraph.cpp:564): Unimplemented Features Pass (OpGraphErrorChecker.cpp:12):
#  backtrace (UnimplementedFeaturesChecker.cpp:376): backtrace (OpGraph.cpp:822):
#  backtrace (OpGraph.cpp:822): backtrace (OpGraph.cpp:798):
#  returning named variable paths not yet supported (UnimplementedFeaturesChecker.cpp:197): InternalError:NotImplemented: InternalError:NotImplemented



In [ ]:

#  How Persons relate to Houston ..
#
#     P_101   Justin
#     P_102   Anand
#     P_103   Brian
#     P_104   Sameer

l_result = my_graph.query("""

   MATCH (n: Person) - [*] - > (m) 
   WHERE
      m.city = 'Houston'
   RETURN *

   """, contextualize=True)

l_result.view()


# display(print(l_result))
# 
#      n.internal_id  n.labels n.LABEL   n.id  n.name n.type  m.internal_id       m.labels      m.LABEL   m.city   m.id m.name m.type
#      
#  0               3  [Person]  Person  P_104  Sameer   node             23     [Reviewer]     Reviewer  Houston  R_102  Allen   node
#  2               3  [Person]  Person  P_104  Sameer   node             21  [Dist_Center]  Dist_Center  Houston  D_102    NaN   node
#  4               3  [Person]  Person  P_104  Sameer   node             21  [Dist_Center]  Dist_Center  Houston  D_102    NaN   node
#  10              3  [Person]  Person  P_104  Sameer   node             19        [Store]        Store  Houston  S_104    NaN   node
#  12              3  [Person]  Person  P_104  Sameer   node             19        [Store]        Store  Houston  S_104    NaN   node
#  
#  1               1  [Person]  Person  P_102   Anand   node             21  [Dist_Center]  Dist_Center  Houston  D_102    NaN   node
#  3               1  [Person]  Person  P_102   Anand   node             21  [Dist_Center]  Dist_Center  Houston  D_102    NaN   node
#  9               1  [Person]  Person  P_102   Anand   node             19        [Store]        Store  Houston  S_104    NaN   node
#  11              1  [Person]  Person  P_102   Anand   node             19        [Store]        Store  Houston  S_104    NaN   node
#  
#  5               0  [Person]  Person  P_101  Justin   node             19        [Store]        Store  Houston  S_104    NaN   node
#  7               0  [Person]  Person  P_101  Justin   node             19        [Store]        Store  Houston  S_104    NaN   node
#  
#  6               2  [Person]  Person  P_103   Brian   node             19        [Store]        Store  Houston  S_104    NaN   node
#  8               2  [Person]  Person  P_103   Brian   node             19        [Store]        Store  Houston  S_104    NaN   node



#  Betweeness algorithm

In [6]:

my_graph.run(lambda g: g.nodes.remove_property("centrality"))

print("--")

#  AttributeError: 'katana_enterprise.distributed_native.Graph' object has no attribute 'nodes'


          0/? [?op/s]


Host 0 errors:
Traceback (most recent call last):
  File "/opt/miniconda/lib/python3.8/site-packages/katana_enterprise/worker/worker.py", line 85, in execute
    value = function(graph)
  File "/tmp/ipykernel_18860/880321459.py", line 1, in <lambda>
AttributeError: 'katana_enterprise.distributed_native.Graph' object has no attribute 'nodes'

Host 1 errors:
Traceback (most recent call last):
  File "/opt/miniconda/lib/python3.8/site-packages/katana_enterprise/worker/worker.py", line 85, in execute
    value = function(graph)
  File "/tmp/ipykernel_18860/880321459.py", line 1, in <lambda>
AttributeError: 'katana_enterprise.distributed_native.Graph' object has no attribute 'nodes'

Host 2 errors:
Traceback (most recent call last):
  File "/opt/miniconda/lib/python3.8/site-packages/katana_enterprise/worker/worker.py", line 85, in execute
    value = function(graph)
  File "/tmp/ipykernel_18860/880321459.py", line 1, in <lambda>
AttributeError: 'katana_enterprise.distributed_native.Graph' 

AttributeError: 'katana_enterprise.distributed_native.Graph' object has no attribute 'nodes'

In [7]:

from katana.remote import analytics

analytics.betweenness_centrality(
   my_graph,
      #
   result_property_name = "centrality",
   num_sources          = 100
   )

print("--")



          0/? [?op/s]

--


In [8]:

l_result = my_graph.query(
   f"""
    
   MATCH (n)
   RETURN n, n.centrality
   ORDER BY n.centrality DESC
   LIMIT 30
    
    """, contextualize=True,
   )

l_result.view()



          0/? [?op/s]

          0/? [?op/s]

NameError: name 'result' is not defined

In [11]:

l_result.view()
